Meu plano de estudos em Python, voltado para Data Science.

O chatGPT e a Alura foram usados como organizadores dos temas de estudo.

Nesse primeiro momento, vamos importar as bibliotecas principais para Data Science

In [116]:
import numpy as np
import pandas as pd
import matplotlib as mp
import requests as rq
import plotly as py
import sqlalchemy as alc
import chardet

url = 'https://raw.githubusercontent.com/alura-cursos/Pandas/main/dados_sus.csv'

Como dados, vamos usar o seguinte arquivo: https://raw.githubusercontent.com/alura-cursos/Pandas/main/dados_sus.csv

Para detectar o tipo de codificação desse arquivo, usaremos o CHARDET

In [117]:
def detectar_codificacao(url):
    resposta = rq.get(url)
    if resposta.status_code == 200:
        conteudo_bytes = resposta.content
        resultado = chardet.detect(conteudo_bytes)
        return resultado['encoding']
    else:
        print(f'Erro ao baixar o arquivo. Código de status: {resposta.status_code}')
        return None
codificacao_detectada = detectar_codificacao(url)

Sabendo a codificação do arquivo, é possível lê-lo e guardá-lo em uma variável.

O index_col=0 indica que a primeira coluna contém os indices das linhas (o nome dos estados)

In [118]:
codificacao_detectada = detectar_codificacao(url)
if (codificacao_detectada!=None):
    dados = pd.read_csv(url, sep=';', skiprows=3, skipfooter=9, encoding=codificacao_detectada, engine='python', index_col=0)
print(dados.head(3))

                        2008/Jan    2008/Fev    2008/Mar    2008/Abr  \
Unidade da Federação                                                   
Rondônia              1388528,39  2931283,42  1541682,52  1525314,96   
Acre                   902416,00  1497206,26  1794028,48  1730469,42   
Amazonas              4735529,42  7118990,57  8196635,49  8259378,42   

                        2008/Mai    2008/Jun    2008/Jul    2008/Ago  \
Unidade da Federação                                                   
Rondônia              1645953,84  1406150,68  3065279,01  3231494,61   
Acre                  1819443,92  1828496,00  2511754,59  2089107,14   
Amazonas              7831399,90  8477273,62  9368858,72  9352532,70   

                        2008/Set    2008/Out  ...     2020/Jul     2020/Ago  \
Unidade da Federação                          ...                             
Rondônia              3117178,63  3068430,44  ...  11824687,13  11733307,76   
Acre                  2275708,53  1923886

Certamente, é uma boa prática realizar uma "sanitização" dos dados antes de começar a trabalhar com eles. Você pode seguir os passos que você mencionou: eliminar valores NaN (se houver) e converter todas as colunas para o tipo `float`, trocando as vírgulas por pontos. Aqui está um exemplo de como você pode fazer isso:

```python
# Eliminar valores NaN
dados = dados.dropna()

# Substituir vírgulas por pontos e converter para float
dados = dados.apply(lambda x: x.str.replace(',', '.').astype(float) if x.dtype == 'O' else x)
```

Este código utiliza o método `dropna` para eliminar linhas que contenham valores NaN e, em seguida, utiliza a função `apply` para percorrer todas as colunas e substituir vírgulas por pontos, convertendo os valores para o tipo `float`. Este processo ajudará a garantir que todas as operações subsequentes sejam realizadas corretamente.

In [119]:
# dados = dados.dropna() // o dropna irá eliminar as linhas que contiverem algum valor NaN
# dados = dados.fillna(0) // o fillna irá substituir as celulas que contiverem valor NaN por 0
# dados = dados.apply(lambda x: x.str.replace(',', '.')) // Substituir vírgulas por pontos
# dados = dados.apply(pd.to_numeric, errors='coerce') // Converter para float usando pd.to_numeric, tratando valores não convertíveis como NaN
# linhas_com_nan = dados[dados.isna().any(axis=1)] // Encontrar linhas que contêm NaN

dados = dados.apply(lambda x: x.str.replace(',', '.'))
dados = dados.apply(pd.to_numeric, errors='coerce')
dados = dados.fillna(0)

Não tenho certeza se a última coluna, com o total, está calculada corretamente. Vamos apagá-la e fazer o cálculo novamente.

Para excluir a última coluna de um DataFrame do Pandas, você pode usar o método `drop` com o argumento `axis=1`. Aqui está um exemplo de como você pode fazer isso:

```python
# Supondo que a última coluna seja chamada 'Total'
dados = dados.drop('Total', axis=1)
```

Certifique-se de ajustar o nome da coluna ('Total' no exemplo acima) para corresponder ao nome real da última coluna que você deseja excluir.

Se você não souber o nome exato da última coluna, você pode usar o método `iloc` para obter o índice da última coluna e, em seguida, usar esse índice no método `drop`:

```python
# Obter o índice da última coluna
indice_ultima_coluna = dados.columns.get_loc(dados.columns[-1])

# Excluir a última coluna usando o índice
dados = dados.drop(dados.columns[indice_ultima_coluna], axis=1)
```

Lembre-se de que essas operações não alteram o DataFrame original. Se você quiser modificar o DataFrame existente, você pode atribuir o resultado de volta à variável `dados`:

```python
dados = dados.drop('Total', axis=1)
# ou
dados = dados.drop(dados.columns[indice_ultima_coluna], axis=1)
```
O parâmetro `axis` no método `drop` do Pandas indica ao método ao longo de qual eixo a operação deve ser realizada. No contexto do método `drop`, `axis=1` refere-se ao eixo das colunas.

- `axis=0`: Remove linhas.
- `axis=1`: Remove colunas.

Quando você utiliza `axis=1`, está especificando que deseja remover colunas. Por exemplo, ao fazer `dados.drop('Total', axis=1)`, você está instruindo o Pandas a eliminar a coluna chamada 'Total'.

Caso você não especifique o parâmetro `axis` (ou utilize `axis=0`, que é o padrão), o Pandas assumirá que você está referindo-se às linhas. Portanto, `dados.drop('Total')` sem o parâmetro `axis` removeria a linha com o índice 'Total' (caso existisse).


In [120]:
dados = dados.drop('Total', axis=1)
print(dados.head(3))

                        2008/Jan    2008/Fev    2008/Mar    2008/Abr  \
Unidade da Federação                                                   
Rondônia              1388528.39  2931283.42  1541682.52  1525314.96   
Acre                   902416.00  1497206.26  1794028.48  1730469.42   
Amazonas              4735529.42  7118990.57  8196635.49  8259378.42   

                        2008/Mai    2008/Jun    2008/Jul    2008/Ago  \
Unidade da Federação                                                   
Rondônia              1645953.84  1406150.68  3065279.01  3231494.61   
Acre                  1819443.92  1828496.00  2511754.59  2089107.14   
Amazonas              7831399.90  8477273.62  9368858.72  9352532.70   

                        2008/Set    2008/Out  ...     2020/Jun     2020/Jul  \
Unidade da Federação                          ...                             
Rondônia              3117178.63  3068430.44  ...   9246097.96  11824687.13   
Acre                  2275708.53  1923886

Agora vamos criar a coluna nova.

Para adicionar uma nova coluna chamada "Resultado" ao final do DataFrame no Pandas, você pode simplesmente atribuir valores à nova coluna. Se você quiser atribuir um valor específico a toda a coluna, pode fazer algo como:

```python
dados['Resultado'] = 0  # Por exemplo, atribuindo o valor 0 a toda a coluna
```

Se você quiser calcular o valor da nova coluna com base em alguma lógica ou em outras colunas, você pode fazer algo como:

```python
# Exemplo: Soma das colunas 'A' e 'B' como valor para a nova coluna 'Resultado'
dados['Resultado'] = dados['A'] + dados['B']
```

In [121]:
# dados['Total'] = dados.sum(axis=1) // Neste exemplo, axis=1 indica que a soma deve ser realizada ao longo das colunas (horizontalmente), criando assim a nova coluna 'Total' que contém a soma das colunas anteriores para cada linha.
dados['Total'] = dados.sum(axis=1)
print(dados.head(3))

                        2008/Jan    2008/Fev    2008/Mar    2008/Abr  \
Unidade da Federação                                                   
Rondônia              1388528.39  2931283.42  1541682.52  1525314.96   
Acre                   902416.00  1497206.26  1794028.48  1730469.42   
Amazonas              4735529.42  7118990.57  8196635.49  8259378.42   

                        2008/Mai    2008/Jun    2008/Jul    2008/Ago  \
Unidade da Federação                                                   
Rondônia              1645953.84  1406150.68  3065279.01  3231494.61   
Acre                  1819443.92  1828496.00  2511754.59  2089107.14   
Amazonas              7831399.90  8477273.62  9368858.72  9352532.70   

                        2008/Set    2008/Out  ...     2020/Jul     2020/Ago  \
Unidade da Federação                          ...                             
Rondônia              3117178.63  3068430.44  ...  11824687.13  11733307.76   
Acre                  2275708.53  1923886



É ótimo ver que você já importou várias bibliotecas essenciais para análise de dados em Python, como NumPy, Pandas, Matplotlib, Plotly e SQLAlchemy. Sua abordagem inicial, incluindo a detecção de codificação, também é uma boa prática. Agora, você pode explorar algumas estruturas e métodos essenciais para análise de dados com Pandas e visualização de dados com Matplotlib e Plotly. Aqui estão algumas sugestões de tópicos e métodos para estudar na sequência:

1. **Manipulação de Dados com Pandas:**
    - **Seleção e Filtragem de Dados:**
      - `loc` e `iloc` para seleção de linhas e colunas.
      - Filtros condicionais.
    
    - **Limpeza de Dados:**
      - Lidar com valores ausentes (`dropna`, `fillna`).
      - Remover duplicatas (`drop_duplicates`).
    
    - **Agregação e Resumo de Dados:**
      - `groupby` para agrupamento.
      - Métodos de agregação (`sum`, `mean`, `count`, etc.).
      - `pivot_table` para criar tabelas dinâmicas.

    - **Operações com Séries Temporais:**
      - Trabalhar com datas e horários.
      - Resampling e shifting.

    - **Concatenação e Junção de DataFrames:**
      - `concat` e `merge` para combinar DataFrames.

2. **Visualização de Dados com Matplotlib:**
    - **Gráficos Básicos:**
      - Gráficos de linha, dispersão e barras.

    - **Customização de Gráficos:**
      - Adição de rótulos, títulos, legendas.
      - Controle de cores e estilos.

3. **Visualização de Dados com Plotly:**
    - **Gráficos Interativos:**
      - Gráficos de dispersão, barras e linha interativos.

    - **Dashboards Interativos:**
      - Construção de dashboards com o Dash.

4. **Conexão com Banco de Dados usando SQLAlchemy:**
    - **Conectar ao Banco de Dados:**
      - Criar uma conexão usando SQLAlchemy.
    
    - **Querying:**
      - Executar consultas SQL usando Pandas e SQLAlchemy.
    
    - **Escrever no Banco de Dados:**
      - Inserir dados em tabelas.

5. **Análise Exploratória de Dados (EDA):**
    - **Histogramas e Estatísticas Descritivas:**
      - Usar `describe`, `hist` para entender a distribuição dos dados.

    - **Correlações e Heatmaps:**
      - Explorar relações entre variáveis.

Estes são apenas alguns pontos iniciais, e à medida que você avança, pode explorar tópicos mais avançados, como machine learning com Scikit-Learn, estatísticas mais avançadas com StatsModels, entre outros, dependendo dos seus interesses e objetivos específicos. A documentação oficial de cada biblioteca é uma excelente fonte de informações e exemplos práticos.

1. **Manipulação de Dados com Pandas:**
    - **Seleção e Filtragem de Dados:**
      - `loc` e `iloc` para seleção de linhas e colunas.
      - Filtros condicionais.

`loc` e `iloc` são métodos principais em Pandas que são usados para acessar e selecionar dados em um DataFrame. Eles são especialmente úteis para fatiar e indexar dados de maneira eficiente. Aqui estão detalhes sobre cada um:

1. **`loc` (Label-based location):**
   - `loc` é usado para selecionar dados com base em rótulos de linhas ou colunas.
   - Sintaxe: `df.loc[linhas, colunas]`.
   - As linhas e colunas podem ser especificadas por rótulos.

   Exemplo:
   ```python
   import pandas as pd

   data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
   df = pd.DataFrame(data, index=['X', 'Y', 'Z'])

   # Selecionar todas as linhas da coluna 'A'
   resultado = df.loc[:, 'A']

   # Selecionar a célula na linha 'Y' e coluna 'B'
   resultado_celula = df.loc['Y', 'B']
   ```

    ```python
    print(df)
    ```

    O resultado será:

    ```
       A  B  C
    X  1  4  7
    Y  2  5  8
    Z  3  6  9
    ```


2. **`iloc` (Integer-location based):**
   - `iloc` é usado para selecionar dados com base em índices de linhas e colunas, ou seja, posições inteiras.
   - Sintaxe: `df.iloc[linhas, colunas]`.
   - As linhas e colunas são especificadas por números inteiros.

   Exemplo:
   ```python
   import pandas as pd

   data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
   df = pd.DataFrame(data)

   # Selecionar todas as linhas da segunda coluna (índice 1)
   resultado = df.iloc[:, 1]

   # Selecionar a célula na segunda linha (índice 1) e terceira coluna (índice 2)
   resultado_celula = df.iloc[1, 2]
   ```

Ambos `loc` e `iloc` suportam operações de fatiamento e aceitam listas ou arrays booleanos como argumentos. Eles são poderosos e flexíveis para acessar dados em DataFrames, permitindo a seleção de linhas e colunas específicas com base em rótulos ou índices. A escolha entre `loc` e `iloc` depende da natureza dos seus dados e da preferência pessoal.

É um ponto comum de confusão para muitas pessoas, especialmente para quem está começando a trabalhar com Pandas. No DataFrame do Pandas, as letras (A, B, C, etc.) referem-se aos nomes das colunas, não das linhas.

Ao criar um DataFrame como:

```python
df = pd.DataFrame(data, index=['X', 'Y', 'Z'])
```

- `'A', 'B', 'C'` são os nomes das colunas (rótulos das colunas).
- `'X', 'Y', 'Z'` são os rótulos das linhas (índices).

Assim, você acessa as colunas usando as letras e as linhas usando os índices. Por exemplo, `df['A']` acessa a coluna 'A', e `df.loc['X']` acessa a linha com o rótulo 'X'.

Esta convenção pode parecer invertida para alguns, mas é consistente com a ideia de que, em geral, você tem várias observações (linhas) para cada variável (coluna) em análise de dados tabulares. Espero que isso esclareça a confusão!

In [122]:
tres_linhas = dados.head(3)
exemplo_loc_coluna = tres_linhas.loc[:,'2008/Jan']
exemplo_loc_coluna

Unidade da Federação
Rondônia    1388528.39
Acre         902416.00
Amazonas    4735529.42
Name: 2008/Jan, dtype: float64

In [123]:
exemplo_loc_linha = tres_linhas.loc['Rondônia',:]
exemplo_loc_linha

2008/Jan    1.388528e+06
2008/Fev    2.931283e+06
2008/Mar    1.541683e+06
2008/Abr    1.525315e+06
2008/Mai    1.645954e+06
                ...     
2020/Dez    8.880837e+06
2021/Jan    9.266015e+06
2021/Fev    7.730597e+06
2021/Mar    1.102331e+07
Total       9.964113e+08
Name: Rondônia, Length: 160, dtype: float64

In [124]:
exemplo_iloc_coluna = tres_linhas.iloc[:,0]
exemplo_iloc_coluna

Unidade da Federação
Rondônia    1388528.39
Acre         902416.00
Amazonas    4735529.42
Name: 2008/Jan, dtype: float64

In [125]:
exemplo_iloc_linha = tres_linhas.iloc[0,:]
exemplo_iloc_linha

2008/Jan    1.388528e+06
2008/Fev    2.931283e+06
2008/Mar    1.541683e+06
2008/Abr    1.525315e+06
2008/Mai    1.645954e+06
                ...     
2020/Dez    8.880837e+06
2021/Jan    9.266015e+06
2021/Fev    7.730597e+06
2021/Mar    1.102331e+07
Total       9.964113e+08
Name: Rondônia, Length: 160, dtype: float64

In [126]:
exemplo_iloc_celula = tres_linhas.iloc[0,0]
exemplo_iloc_celula

1388528.39

In [127]:
exemplo_iloc_sub_data_frame = tres_linhas.iloc[0:2,0:2]
exemplo_iloc_sub_data_frame

,2008/Jan,2008/Fev
Unidade da Federação,,
Rondônia,1388528.39,2931283.42
Acre,902416.00,1497206.26


**Filtros Condicionais em Pandas:**

Os filtros condicionais são uma parte essencial da manipulação de dados em Pandas. Eles permitem que você selecione dados com base em condições específicas. Aqui estão algumas abordagens comuns para aplicar filtros condicionais em um DataFrame:

1. **Filtros Simples:**
   - Você pode usar operadores de comparação (`>`, `<`, `==`, `!=`, etc.) para criar filtros condicionais.
   - Exemplo: Selecionar todas as linhas onde o valor na coluna 'A' é maior que 10.
     ```python
     filtro = df['A'] > 10
     resultado = df[filtro]
     ```

2. **Filtros Combinados:**
   - Você pode combinar múltiplos filtros usando operadores lógicos como `&` (and), `|` (or), e `~` (not).
   - Exemplo: Selecionar todas as linhas onde o valor na coluna 'A' é maior que 10 e na coluna 'B' é menor que 5.
     ```python
     filtro = (df['A'] > 10) & (df['B'] < 5)
     resultado = df[filtro]
     ```

3. **Usando o Método `query`:**
   - O método `query` permite que você escreva expressões de consulta de forma mais expressiva.
   - Exemplo: Selecionar todas as linhas onde o valor na coluna 'A' é maior que 10 usando `query`.
     ```python
     resultado = df.query('A > 10')
     ```

4. **Método `isin`:**
   - Você pode usar o método `isin` para filtrar com base em uma lista de valores.
   - Exemplo: Selecionar todas as linhas onde o valor na coluna 'A' está em uma lista específica.
     ```python
     valores_para_filtrar = [10, 20, 30]
     filtro = df['A'].isin(valores_para_filtrar)
     resultado = df[filtro]
     ```

5. **Usando Métodos de String para Filtros:**
   - Se você estiver lidando com dados de texto, métodos de string como `str.contains` podem ser úteis.
   - Exemplo: Selecionar todas as linhas onde a coluna 'Texto' contém a palavra 'Python'.
     ```python
     filtro = df['Texto'].str.contains('Python')
     resultado = df[filtro]
     ```

6. **Filtros Condicionais com `loc` e `iloc`:**
   - Você também pode aplicar filtros condicionais usando `loc` para seleção baseada em rótulos e `iloc` para seleção baseada em índices.
   - Exemplo: Selecionar todas as linhas onde o valor na coluna 'A' é maior que 10 usando `loc`.
     ```python
     resultado = df.loc[df['A'] > 10, :]
     ```

Estas são apenas algumas das muitas maneiras de aplicar filtros condicionais em Pandas. Escolha a abordagem que melhor se adequa ao seu caso de uso específico.

In [128]:
# filtro = tres_linhas['2008/Jan'] > 100000 // Filtro Simples
filtro = tres_linhas['2008/Jan'] > 100000
resultado = tres_linhas[filtro]
print(resultado)

                        2008/Jan    2008/Fev    2008/Mar    2008/Abr  \
Unidade da Federação                                                   
Rondônia              1388528.39  2931283.42  1541682.52  1525314.96   
Acre                   902416.00  1497206.26  1794028.48  1730469.42   
Amazonas              4735529.42  7118990.57  8196635.49  8259378.42   

                        2008/Mai    2008/Jun    2008/Jul    2008/Ago  \
Unidade da Federação                                                   
Rondônia              1645953.84  1406150.68  3065279.01  3231494.61   
Acre                  1819443.92  1828496.00  2511754.59  2089107.14   
Amazonas              7831399.90  8477273.62  9368858.72  9352532.70   

                        2008/Set    2008/Out  ...     2020/Jul     2020/Ago  \
Unidade da Federação                          ...                             
Rondônia              3117178.63  3068430.44  ...  11824687.13  11733307.76   
Acre                  2275708.53  1923886

In [129]:
# filtro = (df['A'] > 10) & (df['B'] < 5) // Filtro Compbinado
filtro = (tres_linhas['2008/Jan'] > 100000) & (tres_linhas['2008/Fev'] < 5000000)
resultado = tres_linhas[filtro]
print(resultado)

                        2008/Jan    2008/Fev    2008/Mar    2008/Abr  \
Unidade da Federação                                                   
Rondônia              1388528.39  2931283.42  1541682.52  1525314.96   
Acre                   902416.00  1497206.26  1794028.48  1730469.42   

                        2008/Mai    2008/Jun    2008/Jul    2008/Ago  \
Unidade da Federação                                                   
Rondônia              1645953.84  1406150.68  3065279.01  3231494.61   
Acre                  1819443.92  1828496.00  2511754.59  2089107.14   

                        2008/Set    2008/Out  ...     2020/Jul     2020/Ago  \
Unidade da Federação                          ...                             
Rondônia              3117178.63  3068430.44  ...  11824687.13  11733307.76   
Acre                  2275708.53  1923886.57  ...   3915193.20   3640142.82   

                         2020/Set    2020/Out    2020/Nov    2020/Dez  \
Unidade da Federação            

In [130]:
# resultado = df.query('A > 10') // Usando o Método query
consulta = "`2008/Jan` > 100000 & `2008/Fev` < 5000000"
resultado = tres_linhas.query(consulta)
print(resultado)

                        2008/Jan    2008/Fev    2008/Mar    2008/Abr  \
Unidade da Federação                                                   
Rondônia              1388528.39  2931283.42  1541682.52  1525314.96   
Acre                   902416.00  1497206.26  1794028.48  1730469.42   

                        2008/Mai    2008/Jun    2008/Jul    2008/Ago  \
Unidade da Federação                                                   
Rondônia              1645953.84  1406150.68  3065279.01  3231494.61   
Acre                  1819443.92  1828496.00  2511754.59  2089107.14   

                        2008/Set    2008/Out  ...     2020/Jul     2020/Ago  \
Unidade da Federação                          ...                             
Rondônia              3117178.63  3068430.44  ...  11824687.13  11733307.76   
Acre                  2275708.53  1923886.57  ...   3915193.20   3640142.82   

                         2020/Set    2020/Out    2020/Nov    2020/Dez  \
Unidade da Federação            

In [131]:
# Método isin
valores_para_filtrar = [902416.00, 4735529.42]
filtro = tres_linhas['2008/Jan'].isin(valores_para_filtrar)
resultado = tres_linhas[filtro]
print(resultado)

                        2008/Jan    2008/Fev    2008/Mar    2008/Abr  \
Unidade da Federação                                                   
Acre                   902416.00  1497206.26  1794028.48  1730469.42   
Amazonas              4735529.42  7118990.57  8196635.49  8259378.42   

                        2008/Mai    2008/Jun    2008/Jul    2008/Ago  \
Unidade da Federação                                                   
Acre                  1819443.92  1828496.00  2511754.59  2089107.14   
Amazonas              7831399.90  8477273.62  9368858.72  9352532.70   

                        2008/Set    2008/Out  ...     2020/Jul     2020/Ago  \
Unidade da Federação                          ...                             
Acre                  2275708.53  1923886.57  ...   3915193.20   3640142.82   
Amazonas              9363099.35  8618318.83  ...  19769460.14  18059931.43   

                         2020/Set     2020/Out     2020/Nov     2020/Dez  \
Unidade da Federação         

In [132]:
# Usando Métodos de String para Filtros
# Se você estiver lidando com dados de texto, métodos de string como str.contains podem ser úteis.
# Exemplo: Selecionar todas as linhas onde a coluna 'Texto' contém a palavra 'Python'.
# filtro = df['Texto'].str.contains('Python')
# resultado = df[filtro]
# Esse código não funciona aqui, pois o DF contém apenas float

In [133]:
# Filtros Condicionais com loc e iloc:
# Você também pode aplicar filtros condicionais usando loc para seleção baseada em rótulos e iloc para seleção baseada em índices.
# Exemplo: Selecionar todas as linhas onde o valor na coluna '2008/Jan' é maior que 1000000 usando loc.
resultado = tres_linhas.loc[tres_linhas['2008/Jan'] > 1000000, '2008/Jan':'2008/Mar']
print(resultado)

                        2008/Jan    2008/Fev    2008/Mar
Unidade da Federação                                    
Rondônia              1388528.39  2931283.42  1541682.52
Amazonas              4735529.42  7118990.57  8196635.49


**Limpeza de Dados:**
  - Lidar com valores ausentes (`dropna`, `fillna`).
  - Remover duplicatas (`drop_duplicates`).

As funções de dropna e fillna já usamos para a sanitização inicial do DF.

Agora vamos pesquisar se existem duplicatas.

A função `drop_duplicates` no Pandas é utilizada para remover linhas duplicadas de um DataFrame. Ele retorna um novo DataFrame com as linhas duplicadas removidas. Vamos explorar alguns detalhes sobre como usar essa função:

### Sintaxe básica:
```python
novo_dataframe = dataframe.drop_duplicates()
```

### Parâmetros principais:
- **subset:** Especifica as colunas nas quais procurar duplicatas. Pode ser uma única coluna ou uma lista de colunas.
  ```python
  novo_dataframe = dataframe.drop_duplicates(subset=['coluna1', 'coluna2'])
  ```

- **keep:** Controla qual instância de linhas duplicadas manter. Pode ser:
  - `'first'` (padrão): Mantém a primeira instância e remove as subsequentes.
  - `'last'`: Mantém a última instância e remove as anteriores.
  - `False`: Remove todas as instâncias duplicadas.
  ```python
  novo_dataframe = dataframe.drop_duplicates(keep='last')
  ```

- **inplace:** Se definido como `True`, modifica o DataFrame original e retorna `None`. Se definido como `False` (padrão), retorna um novo DataFrame sem modificar o original.
  ```python
  dataframe.drop_duplicates(inplace=True)
  ```

### Exemplo:
```python
import pandas as pd

# Criar um DataFrame de exemplo
dados = {'A': [1, 2, 3, 1, 2],
         'B': ['x', 'y', 'z', 'x', 'y'],
         'C': [10, 20, 30, 10, 20]}

df = pd.DataFrame(dados)

# Remover linhas duplicadas
df_sem_duplicatas = df.drop_duplicates()

# Exibir o resultado
print(df_sem_duplicatas)
```

Neste exemplo, `df_sem_duplicatas` será um novo DataFrame sem as linhas duplicadas do DataFrame original `df`. Se houver linhas completamente idênticas, apenas a primeira delas será mantida por padrão.

In [136]:
dados = dados.drop_duplicates()

**Agregação e Resumo de Dados:**
  - `groupby` para agrupamento.
  - Métodos de agregação (`sum`, `mean`, `count`, etc.).
  - `pivot_table` para criar tabelas dinâmicas.

O método `groupby` no Pandas é utilizado para agrupar dados com base em uma ou mais colunas. Ele é frequentemente combinado com operações de agregação para realizar cálculos específicos em grupos de dados. Aqui estão alguns detalhes sobre como usar `groupby`:

### Sintaxe básica:
```python
grupo = dataframe.groupby('coluna')
```

### Agrupamento por múltiplas colunas:
```python
grupo = dataframe.groupby(['coluna1', 'coluna2'])
```

### Iteração sobre grupos:
```python
for nome_grupo, grupo_dados in dataframe.groupby('coluna'):
    # nome_grupo é o valor único da coluna usada para o agrupamento
    # grupo_dados é um DataFrame contendo todas as linhas correspondentes ao valor do grupo
    # (nome_grupo, grupo_dados) podem ser usados para realizar operações específicas em cada grupo
```

### Operações de agregação:
```python
# Exemplo: Calcular a média para cada grupo
media_por_grupo = dataframe.groupby('coluna').mean()

# Outras operações de agregação comuns: sum(), min(), max(), count(), etc.
```

### Aplicar funções personalizadas:
```python
# Exemplo: Calcular a soma dos valores elevados ao quadrado para cada grupo
def soma_quadrados(grupo):
    return grupo['coluna'].apply(lambda x: x**2).sum()

resultado = dataframe.groupby('outra_coluna').apply(soma_quadrados)
```

### Agregar com base em várias funções:
```python
# Exemplo: Calcular a média e a soma para cada grupo
resultados = dataframe.groupby('coluna').agg({'coluna1': 'mean', 'coluna2': 'sum'})
```

### Resetar o índice após o agrupamento:
```python
grupo = dataframe.groupby('coluna').mean().reset_index()
```

O método `groupby` permite uma poderosa manipulação e análise de dados em conjuntos grandes, permitindo que você faça cálculos específicos para cada grupo de dados. Certifique-se de adaptar as operações de agrupamento às suas necessidades específicas.

In [144]:
grupo_teste = dados.groupby('2008/Jan').count()
print(grupo_teste)

              2008/Fev  2008/Mar  2008/Abr  2008/Mai  2008/Jun  2008/Jul  \
2008/Jan                                                                   
6.578895e+05         1         1         1         1         1         1   
9.024160e+05         1         1         1         1         1         1   
1.120973e+06         1         1         1         1         1         1   
1.388528e+06         1         1         1         1         1         1   
2.780750e+06         1         1         1         1         1         1   
3.441593e+06         1         1         1         1         1         1   
4.735529e+06         1         1         1         1         1         1   
4.745919e+06         1         1         1         1         1         1   
4.795299e+06         1         1         1         1         1         1   
5.377392e+06         1         1         1         1         1         1   
5.456724e+06         1         1         1         1         1         1   
5.947814e+06

Os métodos de agregação no Pandas são usados em conjunto com o `groupby` para realizar operações específicas em grupos de dados. Aqui estão alguns dos métodos de agregação comuns:

### Média (`mean()`):
```python
# Calcula a média para cada grupo
media_por_grupo = dataframe.groupby('coluna').mean()
```

### Soma (`sum()`):
```python
# Calcula a soma para cada grupo
soma_por_grupo = dataframe.groupby('coluna').sum()
```

### Contagem (`count()`):
```python
# Conta o número de observações em cada grupo
contagem_por_grupo = dataframe.groupby('coluna').count()
```

### Mínimo (`min()`):
```python
# Encontra o valor mínimo em cada grupo
minimo_por_grupo = dataframe.groupby('coluna').min()
```

### Máximo (`max()`):
```python
# Encontra o valor máximo em cada grupo
maximo_por_grupo = dataframe.groupby('coluna').max()
```

### Mediana (`median()`):
```python
# Calcula a mediana para cada grupo
mediana_por_grupo = dataframe.groupby('coluna').median()
```

### Desvio padrão (`std()`):
```python
# Calcula o desvio padrão para cada grupo
std_por_grupo = dataframe.groupby('coluna').std()
```

### Percentis (`quantile()`):
```python
# Calcula percentis para cada grupo (por exemplo, 25%, 50%, 75%)
percentis_por_grupo = dataframe.groupby('coluna').quantile([0.25, 0.5, 0.75])
```

Estes são apenas alguns exemplos de métodos de agregação disponíveis. Você pode personalizar ainda mais suas operações de agregação usando funções personalizadas. Além disso, a função `agg()` permite que você aplique diferentes funções de agregação a diferentes colunas.

```python
# Aplicar média a uma coluna e soma a outra
resultados = dataframe.groupby('coluna').agg({'coluna1': 'mean', 'coluna2': 'sum'})
```

Adaptar as operações de agregação às suas necessidades específicas é essencial para explorar e entender seus dados de maneira eficaz.

In [145]:
print(dados.groupby('Total').mean())

                  2008/Jan      2008/Fev      2008/Mar      2008/Abr  \
Total                                                                  
3.098601e+08  1.120973e+06  1.268740e+06  9.746512e+05  1.141137e+06   
3.288770e+08  6.578895e+05  7.779393e+05  7.186880e+05  8.399944e+05   
4.500485e+08  9.024160e+05  1.497206e+06  1.794028e+06  1.730469e+06   
9.964113e+08  1.388528e+06  2.931283e+06  1.541683e+06  1.525315e+06   
1.020028e+09  3.441593e+06  4.985689e+06  6.205613e+06  4.984482e+06   
1.215853e+09  2.780750e+06  3.676483e+06  5.051474e+06  5.046063e+06   
1.917248e+09  4.735529e+06  7.118991e+06  8.196635e+06  8.259378e+06   
2.180120e+09  4.795299e+06  8.684882e+06  7.886243e+06  8.794110e+06   
2.236415e+09  6.243726e+06  8.376622e+06  8.408336e+06  9.090795e+06   
2.280889e+09  7.985142e+06  1.007993e+07  1.029837e+07  1.077598e+07   
2.324525e+09  5.377392e+06  7.002473e+06  8.423933e+06  8.797488e+06   
2.592423e+09  4.745919e+06  7.632481e+06  9.647203e+06  1.109873

A função `pivot_table` no Pandas é uma poderosa ferramenta para criar tabelas dinâmicas, que são uma forma de resumir e analisar dados em formato de tabela. Ela permite que você agregue e organize dados de maneira flexível. Aqui estão alguns detalhes sobre como usar `pivot_table`:

### Sintaxe básica:
```python
tabela_dinamica = pd.pivot_table(dataframe, values='valor', index='coluna_indice', columns='coluna_coluna', aggfunc='funcao_agregacao')
```

- `dataframe`: O DataFrame contendo os dados.
- `values`: A coluna cujos valores serão agregados.
- `index`: As colunas que serão usadas como índices (linhas) na tabela dinâmica.
- `columns`: As colunas que serão usadas como colunas na tabela dinâmica.
- `aggfunc`: A função de agregação a ser aplicada (por padrão, é a média).

### Exemplo:
```python
import pandas as pd

# Criar um DataFrame de exemplo
dados = {'Produto': ['A', 'B', 'A', 'B', 'A', 'B'],
         'Vendedor': ['Alice', 'Bob', 'Alice', 'Bob', 'Alice', 'Bob'],
         'Vendas': [100, 200, 150, 250, 120, 180]}

df = pd.DataFrame(dados)

# Criar uma tabela dinâmica com média de vendas por produto e vendedor
tabela_dinamica = pd.pivot_table(df, values='Vendas', index='Produto', columns='Vendedor', aggfunc='mean')

# Exibir a tabela dinâmica
print(tabela_dinamica)
```

O resultado seria:

```
Vendedor  Alice  Bob
Produto            
A          135   200
B          NaN   215
```

Neste exemplo, a tabela dinâmica mostra a média de vendas para cada combinação de Produto e Vendedor. Se um valor não estiver disponível para uma combinação específica, será exibido como `NaN`.

### Lidando com valores ausentes:
Você pode usar o parâmetro `fill_value` para substituir os valores ausentes por um valor específico:

```python
tabela_dinamica = pd.pivot_table(df, values='Vendas', index='Produto', columns='Vendedor', aggfunc='mean', fill_value=0)
```

### Múltiplas funções de agregação:
Você pode passar uma lista de funções para o parâmetro `aggfunc` para calcular várias estatísticas ao mesmo tempo:

```python
tabela_dinamica = pd.pivot_table(df, values='Vendas', index='Produto', columns='Vendedor', aggfunc=['mean', 'sum'])
```

A função `pivot_table` é muito flexível e pode ser personalizada de acordo com suas necessidades específicas. Experimente diferentes combinações de parâmetros para atender aos seus requisitos de análise de dados.

**Operações com Séries Temporais:**
  - Trabalhar com datas e horários.
  - Resampling e shifting.

**Concatenação e Junção de DataFrames:**
  - `concat` e `merge` para combinar DataFrames.

Operações com Séries Temporais no Pandas são especialmente poderosas e flexíveis. Vamos explorar alguns detalhes sobre como trabalhar com datas e horários, bem como resampling e shifting.

### Trabalhando com Datas e Horários:

#### Criando uma Série Temporal com Datas:
```python
import pandas as pd

# Criar um intervalo de datas
datas = pd.date_range('2022-01-01', '2022-01-10', freq='D')

# Criar uma Série Temporal com datas como índice
serie_temporal = pd.Series(range(len(datas)), index=datas)
```

#### Acessando Componentes de Data:
```python
# Acessar o ano, mês e dia
serie_temporal.index.year
serie_temporal.index.month
serie_temporal.index.day
```

#### Filtrando por Período:
```python
# Filtrar por mês ou ano específico
serie_temporal['2022-01']
serie_temporal['2022']
```

### Resampling e Shifting:

#### Resampling para Frequências Diferentes:
```python
# Resample para frequência mensal, calculando a média
serie_resample = serie_temporal.resample('M').mean()
```

#### Shifting (Lag e Lead):
```python
# Shift para trás (Lag) e para frente (Lead) no tempo
serie_temporal.shift(1)  # Lag de 1 período
serie_temporal.shift(-1)  # Lead de 1 período
```

#### Resampling com Funções de Agregação Personalizadas:
```python
# Resample para média, mas usando uma função de agregação personalizada
def minha_funcao_agregacao(array):
    return array.sum() / len(array)

serie_resample_personalizada = serie_temporal.resample('M').apply(minha_funcao_agregacao)
```

### Lidando com Dados Ausentes:

#### Preenchendo Dados Ausentes:
```python
# Preencher dados ausentes usando interpolação linear
serie_temporal_interpolada = serie_temporal.interpolate()
```

#### Removendo Dados Ausentes:
```python
# Remover linhas com dados ausentes
serie_temporal_sem_nan = serie_temporal.dropna()
```

### Visualização de Séries Temporais:

```python
import matplotlib.pyplot as plt

# Plotar a Série Temporal
serie_temporal.plot(figsize=(10, 6))
plt.title('Série Temporal')
plt.xlabel('Data')
plt.ylabel('Valor')
plt.show()
```

Esses são apenas alguns exemplos de operações que você pode realizar em Séries Temporais com o Pandas. A biblioteca oferece muitas funcionalidades adicionais para análise e manipulação de dados temporais. Ao trabalhar com Séries Temporais, é útil explorar a documentação do Pandas para obter detalhes específicos sobre as funções mencionadas aqui e outras funcionalidades disponíveis.

In [146]:
dados

,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,2008/Out,...,2020/Jul,2020/Ago,2020/Set,2020/Out,2020/Nov,2020/Dez,2021/Jan,2021/Fev,2021/Mar,Total
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,
Rondônia,1.388528e+06,2.931283e+06,1.541683e+06,1.525315e+06,1.645954e+06,1.406151e+06,3.065279e+06,3.231495e+06,3.117179e+06,3.068430e+06,...,1.182469e+07,1.173331e+07,1.020199e+07,7.955139e+06,9.357946e+06,8.880837e+06,9.266015e+06,7.730597e+06,1.102331e+07,9.964113e+08
Acre,9.024160e+05,1.497206e+06,1.794028e+06,1.730469e+06,1.819444e+06,1.828496e+06,2.511755e+06,2.089107e+06,2.275709e+06,1.923887e+06,...,3.915193e+06,3.640143e+06,3.391242e+06,4.044321e+06,3.276590e+06,3.696997e+06,3.715723e+06,3.538428e+06,4.077046e+06,4.500485e+08
Amazonas,4.735529e+06,7.118991e+06,8.196635e+06,8.259378e+06,7.831400e+06,8.477274e+06,9.368859e+06,9.352533e+06,9.363099e+06,8.618319e+06,...,1.976946e+07,1.805993e+07,1.784102e+07,1.640832e+07,1.989562e+07,1.776517e+07,2.143029e+07,2.591713e+07,2.203218e+07,1.917248e+09
Roraima,6.578895e+05,7.779393e+05,7.186880e+05,8.399944e+05,8.623480e+05,8.324407e+05,9.966931e+05,8.942712e+05,9.104242e+05,1.191770e+06,...,3.015488e+06,2.826486e+06,2.928044e+06,3.090314e+06,3.621031e+06,3.454461e+06,3.266928e+06,3.519774e+06,3.985530e+06,3.288770e+08
Pará,1.886474e+07,1.955376e+07,2.193734e+07,2.084283e+07,2.324995e+07,2.324069e+07,2.400222e+07,2.334122e+07,2.517226e+07,2.401171e+07,...,4.080413e+07,4.438572e+07,3.682025e+07,3.696593e+07,3.900432e+07,3.801515e+07,3.835468e+07,3.768831e+07,3.327639e+07,4.705309e+09
Amapá,1.120973e+06,1.268740e+06,9.746512e+05,1.141137e+06,8.663462e+05,1.440979e+06,1.426659e+06,1.475551e+06,1.651341e+06,1.866749e+06,...,3.253399e+06,2.946398e+06,2.416019e+06,2.203732e+06,2.218532e+06,2.566920e+06,2.802886e+06,2.817849e+06,3.188211e+06,3.098601e+08
Tocantins,3.441593e+06,4.985689e+06,6.205613e+06,4.984482e+06,5.511002e+06,4.927934e+06,5.445602e+06,5.413710e+06,5.596670e+06,4.658414e+06,...,9.155572e+06,8.390268e+06,8.729632e+06,7.770886e+06,5.278405e+05,6.752771e+05,4.195535e+05,3.191609e+05,1.357751e+06,1.020028e+09
Maranhão,1.072351e+07,1.355968e+07,1.398514e+07,1.480942e+07,1.450573e+07,1.490809e+07,1.578490e+07,1.519531e+07,1.495128e+07,1.607222e+07,...,3.547349e+07,3.221904e+07,3.007715e+07,3.320078e+07,3.194267e+07,3.078340e+07,3.073875e+07,3.092448e+07,3.966368e+07,3.871069e+09
Piauí,7.985142e+06,1.007993e+07,1.029837e+07,1.077598e+07,1.039836e+07,1.052825e+07,1.104011e+07,1.054918e+07,1.089920e+07,1.171594e+07,...,1.990857e+07,2.566354e+07,2.023078e+07,2.238922e+07,1.891160e+07,2.088397e+07,2.268939e+07,1.940027e+07,2.001833e+07,2.280889e+09


In [151]:
datas = pd.date_range('2008-01', '2021-04', freq='M')
print(datas)

DatetimeIndex(['2008-01-31', '2008-02-29', '2008-03-31', '2008-04-30',
               '2008-05-31', '2008-06-30', '2008-07-31', '2008-08-31',
               '2008-09-30', '2008-10-31',
               ...
               '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30',
               '2020-10-31', '2020-11-30', '2020-12-31', '2021-01-31',
               '2021-02-28', '2021-03-31'],
              dtype='datetime64[ns]', length=159, freq='M')
